这个是寻找最近均线一直上涨的股票

In [1]:
import pandas as pd
import numpy as np
import talib
# import tqdm
from tqdm.notebook import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15.0, 8.0) # 单位是inches

data = pd.read_csv("../../数据获取/聚宽数据/最新数据.csv")
data=data.set_index(['Unnamed: 0', 'Unnamed: 1'])
print('共导入{}行数据,股票个数：{},平均每个股票的的数据量:{}'.format(
    len(data),
    len(data.index.levels[0]),
    len(data)//len(data.index.levels[0])
))
data.head()

共导入3715848行数据,股票个数：4149,平均每个股票的的数据量:895


date   open   high    low  close
Unnamed: 0  Unnamed: 1                                        
000001.XSHE 0           2018-04-20  10.94  11.01  10.65  10.79
            1           2018-04-23  10.74  11.04  10.70  11.00
            2           2018-04-24  11.06  11.35  11.01  11.28
            3           2018-04-25  11.18  11.23  11.06  11.10
            4           2018-04-26  11.08  11.11  10.75  10.86

In [2]:
def find_runs(x):
    """找到连续的"""
    x = np.asanyarray(x) # 首先转成数组
    if x.ndim != 1:
        # 这里只是支持1维的数组
        raise ValueError('only 1D array supported')
    n = x.shape[0] # 维度，这里可以理解成这个一维的长度
    # 
    loc_run_start = np.empty(n, dtype=bool)
    loc_run_start[0] = True 
    loc_run_start # 这个上边构造的是随机的，真正赋值的只有第一项
    # 比较
    np.not_equal(x[:-1], x[1:], out=loc_run_start[1:]) # 这个是将这游戏昂跟下一项比较，如果不同是True，结果放在loc_run_start
    # 相同的是False，不同的是True，这里取得的是不同的位置
    run_starts = np.nonzero(loc_run_start)[0]
    # 这个是值
    run_values = x[loc_run_start]
    # 长度
    run_lengths = np.diff(np.append(run_starts, n))
    return run_starts, run_values, run_lengths

In [4]:
lst_up = [] # 这个是一直上涨的
with tqdm(range(len(data.index.levels[0]))) as bar:
    for id in data.index.levels[0]:
        data_one = data.loc[id].copy() # 这个是每一个股票的数据
        # 上边的格式比较固定，遍历每一个股票,然后下边开始统计
        # 这里首先添加均线，并且只是最近的10天而已。
        for ma_day in [5,10,20,30,60,120]:
            # 插入多条均线
            data_one.insert(
                loc=len(data_one.columns),
                column='ma{}'.format(ma_day),
                value=talib.MA(data_one['close'], timeperiod=ma_day)
            )
        # 我这里只是比较120日均线吧
        data_one.insert(
                loc=len(data_one.columns),
                column='ma120-2',
                value=data_one['ma120'].shift(fill_value=0) # 
            )
        # 然后判断这一天跟前一天的大小
        data_one.insert(
                loc=len(data_one.columns),
                column='涨跌',
                value= data_one['ma120'] >  data_one['ma120-2']# 
            )
        # 
        run_starts, run_values, run_lengths = find_runs(data_one['涨跌'])
        if run_values[-1] and run_lengths[-1] >= 30:
            lst_up.append((id, run_lengths[-1]))
        bar.update(1)

  0%|          | 0/4149 [00:00<?, ?it/s]

In [8]:
lst_up.sort(key=lambda x:x[1])
lst_up.reverse()

In [10]:
lst_up

[('601088.XSHG', 437),
 ('603177.XSHG', 326),
 ('600860.XSHG', 305),
 ('002060.XSHE', 284),
 ('601666.XSHG', 273),
 ('603032.XSHG', 234),
 ('002865.XSHE', 230),
 ('002374.XSHE', 229),
 ('002323.XSHE', 227),
 ('603896.XSHG', 226),
 ('002349.XSHE', 224),
 ('002003.XSHE', 222),
 ('603399.XSHG', 221),
 ('002694.XSHE', 221),
 ('300075.XSHE', 220),
 ('300776.XSHE', 219),
 ('601107.XSHG', 218),
 ('601199.XSHG', 216),
 ('600982.XSHG', 216),
 ('603398.XSHG', 204),
 ('300668.XSHE', 203),
 ('000819.XSHE', 203),
 ('000815.XSHE', 201),
 ('002265.XSHE', 193),
 ('600873.XSHG', 189),
 ('300446.XSHE', 189),
 ('002943.XSHE', 188),
 ('603949.XSHG', 183),
 ('002779.XSHE', 179),
 ('600233.XSHG', 176),
 ('603665.XSHG', 163),
 ('300398.XSHE', 160),
 ('600900.XSHG', 157),
 ('603169.XSHG', 154),
 ('002731.XSHE', 154),
 ('000498.XSHE', 154),
 ('002682.XSHE', 153),
 ('600153.XSHG', 152),
 ('300498.XSHE', 152),
 ('600476.XSHG', 147),
 ('300071.XSHE', 147),
 ('002639.XSHE', 147),
 ('002330.XSHE', 147),
 ('688123.X

In [12]:
# 我这里只是查看价格不到20的
lst_up_2 = []
for i in lst_up:
    data_one =  data.loc[i[0]]
    if data_one.loc[len(data_one)-1, 'close'] <= 20:
        lst_up_2.append(i)

In [14]:
len(lst_up_2)

382

In [15]:
lst_up_2

[('600860.XSHG', 305),
 ('002060.XSHE', 284),
 ('601666.XSHG', 273),
 ('002374.XSHE', 229),
 ('002323.XSHE', 227),
 ('002349.XSHE', 224),
 ('002003.XSHE', 222),
 ('002694.XSHE', 221),
 ('300075.XSHE', 220),
 ('601107.XSHG', 218),
 ('601199.XSHG', 216),
 ('600982.XSHG', 216),
 ('300668.XSHE', 203),
 ('000819.XSHE', 203),
 ('000815.XSHE', 201),
 ('002265.XSHE', 193),
 ('600873.XSHG', 189),
 ('300446.XSHE', 189),
 ('603169.XSHG', 154),
 ('002731.XSHE', 154),
 ('000498.XSHE', 154),
 ('002682.XSHE', 153),
 ('600153.XSHG', 152),
 ('300498.XSHE', 152),
 ('600476.XSHG', 147),
 ('300071.XSHE', 147),
 ('002639.XSHE', 147),
 ('002330.XSHE', 147),
 ('603335.XSHG', 144),
 ('300732.XSHE', 144),
 ('600403.XSHG', 143),
 ('002101.XSHE', 143),
 ('000753.XSHE', 142),
 ('603668.XSHG', 141),
 ('600561.XSHG', 137),
 ('300199.XSHE', 137),
 ('002800.XSHE', 137),
 ('002037.XSHE', 137),
 ('000622.XSHE', 137),
 ('600048.XSHG', 136),
 ('600866.XSHG', 135),
 ('600668.XSHG', 135),
 ('600220.XSHG', 135),
 ('000056.X